# SyftBox SDK Development Testing

This notebook is for testing and validating the SyftBox SDK during development.

In [1]:
%%capture output --no-stderr

# Install the SDK in development mode (run this once)
!pip install nest_asyncio
!pip install -e .

In [2]:
# Import the SDK
import syft_nsai_sdk as sdk
import nest_asyncio
import httpx
import logging
import os

In [3]:
# import client
from syft_nsai_sdk import SyftBoxClient
from syft_nsai_sdk.utils.logger import get_logger
from IPython.display import display, HTML, Markdown

In [4]:
# define async
nest_asyncio.apply()

# initialize logger
logger = get_logger()
logger.success("Logger initialized successfully!")

# Turn off debug logging for your SDK
logging.getLogger('syft_nsai_sdk').setLevel(logging.INFO)

2025-08-29 18:34:39 - syft_nsai_sdk - SUCCESS - Logger initialized successfully!


In [5]:
# initialize client
client = SyftBoxClient()
logger.info(f"Syftbox client created: {client}")

2025-08-29 18:34:40 - syft_nsai_sdk.main - INFO - Found existing accounting credentials in environment
2025-08-29 18:34:40 - syft_nsai_sdk.main - INFO - SyftBoxClient initialized for guest@syft.org
2025-08-29 18:34:40 - syft_nsai_sdk - INFO - Syftbox client created: <syft_nsai_sdk.main.SyftBoxClient object at 0x1094e54c0>


In [ ]:
# # get env vars
# email = os.getenv('SYFTBOX_ACCOUNTING_EMAIL')
# password = os.getenv('SYFTBOX_ACCOUNTING_PASSWORD')
# logger.info(f"SYFTBOX_ACCOUNTING_EMAIL: {email}")
# logger.info(f"SYFTBOX_ACCOUNTING_PASSWORD: {bool(password)}")

# # Setup accounting for paid models
# await client.setup_accounting(email, password)

In [6]:
print(await client.get_account_info())

{'email': 'callis@openmined.org', 'balance': 113.2, 'currency': 'USD', 'account_id': 'cme1ec567000aqn3z9vu7q3st', 'created_at': None}


In [7]:
# Discover models explicitly 
models = client.discover_models(service_type="chat")
print(f"Found {len(models)} models")

2025-08-29 18:34:46 - syft_nsai_sdk.discovery.scanner - INFO - Found 12 models across 256 datasites
2025-08-29 18:34:46 - syft_nsai_sdk.discovery.scanner - INFO - Cached 12 models from 10 owners
2025-08-29 18:34:52 - syft_nsai_sdk.services.health - INFO - Batch health check completed in 6.03s for 8 models
2025-08-29 18:34:52 - syft_nsai_sdk.main - INFO - Discovered 8 models (health_check=True)


Found 8 models


In [8]:
print(f"{'Name':<20} {'Owner':<25} {'Tags':<25} {'Services':<25}")
print("-" * 100)
for model in models:
    enabled_services = [s.type.value for s in model.services if s.enabled]
    tags_str = ", ".join(model.tags[:2])
    services_str = ", ".join(enabled_services)
    print(f"{model.name:<20} {model.owner:<25} {tags_str:<25} {services_str:<25}")

Name                 Owner                     Tags                      Services                 
----------------------------------------------------------------------------------------------------
keelan-test-3        kj@kj.dev                 test                      chat, search             
pal-8                afdolriski@gmail.com                                chat, search             
carl-model           callis@openmined.org      Chat, Search              chat, search             
carl-free            callis@openmined.org      chat, search              chat, search             
claude-sonnet-3.5    aggregator@openmined.org  claude, anthropic         chat                     
deepseek-v3          aggregator@openmined.org  deepseek, opensource      chat                     
test                 irina@openmined.org       test                      chat, search             
sota-model           callisezenwaka@barestripe.com chat, search              chat, search             


In [11]:
result = await client.search(
    model_name="test",
    owner="irina@openmined.org", 
    query="Federated learning",
    limit=10,
    similarity_threshold=0.8
)

2025-08-29 19:37:56 - syft_nsai_sdk.main - INFO - Discovered 1 models (health_check=False)


✓ Response received                                                             



PollingTimeoutError: RPC_ERROR: Polling timed out after 30/30 attempts

In [12]:
# show response
logger.info(f"Search result:\n\n {result}")

NameError: name 'result' is not defined

In [ ]:
# Search multiple models
results = await client.search_multiple_models(
    model_names=["docs", "wiki", "policies"],
    query="vacation policy",
    limit_per_model=3,
    total_limit=10,
    remove_duplicates=True
)

In [ ]:
# show response
logger.info(f"Search results:\n\n {results}")